In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install llama-index langchain
!pip install accelerate sentence-transformers transformers
!pip install optimum[exporters]
!pip install datasets
!pip install llama-hub
!pip install openai

  Obtaining dependency information for llama-index from https://files.pythonhosted.org/packages/42/ad/6d718547e3bccb7aef60d6cfe81459eb8c8285db06516864b7bb6b7b4348/llama_index-0.9.32-py3-none-any.whl.metadata
  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/e8/67/17a732ee99a7a383b5fd51ca671030e9f9d22e6e85af8873d20e2e01f7fd/langchain-0.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp<4.0.0,>=3.8.6 from https://files.pythonhosted.org/packages/2f/16/50441c4baa39e5426181c6f630203ab65029f9a9c55d0a1019a31c26d702/aiohttp-3.9.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for httpx from https://files.pythonhosted.org/packages/39/9b/4937d841aee9c2c8102d9a4eeb800c7dad25386caabb4a1bf5010df81a57/httpx-0.26.0-py3-none-any.whl.metadata
  Obtaining dependency information for nest-asyncio<2.0.0,>=1.5.8 from https://files.pythonhosted.org/packages/0d/d0/ffa604feb4992d5ecb3a0

In [4]:
!pip install 'pydantic<1.8.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.12
    Uninstalling pydantic-1.10.12:
      Successfully uninstalled pydantic-1.10.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.1 requires aiohttp<4.0.0,>=3.8.3, which is not installed.
openai 1.8.0 requires pydantic<3,>=1.9.0, but you have pydantic 1.7.4 which is incompatible.
vaex-core 4.17.1 requires pydantic>=1.8.0, but you have pydantic 1.7.4 which is incompatible.
ydata-profiling 4.5.1 requires numpy<1.24,>=1.16.0, but you have numpy 1.24.3 which is incompatible.
ydata-profiling 4.5.1 requires pydantic<2,>=1.8.1, but you have pydantic 1.7.4 which is incompatible.


In [4]:
# !unzip -d /kaggle/input/wiki-qa/ /kaggle/input/data_raw10k.zip

In [5]:
from llama_index import (
    VectorStoreIndex,
    SummaryIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    GPTVectorStoreIndex,
)
from llama_index.response.notebook_utils import display_response
import torch

In [6]:
import os
from tqdm import tqdm
from llama_index import Document

data_dir = "/kaggle/input/wiki-qa/data_raw10k/"
filenames = sorted(os.listdir(data_dir))[:100]

doc_text = []
for filename in tqdm(filenames):
    filepath = os.path.join(data_dir, filename)
    title = filename.strip(".txt")
    with open(filepath, "r") as f:
        text = f.read()
        text = text.lstrip(title).strip()
        doc_text.append(text)
doc_text = "\n\n".join(doc_text)
docs = [Document(text=doc_text)]

100%|██████████| 100/100 [00:00<00:00, 120.36it/s]


In [7]:
print(docs[0])

Doc ID: c84a9410-feb5-4b0f-824a-3df7f7330853
Text: 1. FC Slovácko là một câu lạc bộ bóng đá Séc có trụ sở ở Uherské
Hradiště. Đội được thành lập năm 1927 với tên gọi SK Staré Město và
vào ngày 1 tháng 7 năm 2000 có tên là 1. FC Synot, với sự hợp nhất của
đội bóng gốc và "FC Slovácká Slavia Uherské Hradiště". Kể từ năm 2009
câu lạc bộ thi đấu ở Giải bóng đá vô địch quốc gia Séc. Slovácko có 2
lần...


In [8]:
from llama_index.llms import HuggingFaceLLM
from llama_index.embeddings import OptimumEmbedding
from langchain.embeddings import HuggingFaceEmbeddings

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    tokenizer_name="vilm/vinallama-7b",
    model_name="vilm/vinallama-7b",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

embed_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")


config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

In [9]:
# initialize service context (set chunk size)
chunk_sizes = [256, 512, 1024]
service_contexts = []
nodes_list = []
vector_indices = []
query_engines = []
for chunk_size in chunk_sizes:
    print(f"Chunk Size: {chunk_size}")
    service_context = ServiceContext.from_defaults(
        chunk_size=chunk_size,
        llm=llm,
        embed_model=embed_model,
    )
    service_contexts.append(service_context)
    nodes = service_context.node_parser.get_nodes_from_documents(docs)

    # add chunk size to nodes to track later
    for node in nodes:
        node.metadata["chunk_size"] = chunk_size
        node.excluded_embed_metadata_keys = ["chunk_size"]
        node.excluded_llm_metadata_keys = ["chunk_size"]

    nodes_list.append(nodes)

    # build vector index
    vector_index = VectorStoreIndex(nodes, service_context=service_context)
    vector_indices.append(vector_index)

    # query engines
    query_engines.append(vector_index.as_query_engine())

Chunk Size: 256


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
response = query_engines[0].query(
    "Nói cho tôi biết về 212 Warrior"
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [11]:
display_response(
    response, show_source=True, source_length=500, show_source_metadata=True
)

**`Final Response:`** 212 Warrior (tên tiếng Việt: Chiến binh 212) (còn được biến đến với tên gọi khác là Wiro Sableng 212 và Wiro Sableng Pendekar Kapak Maut Naga Geni 212) là một bộ phim hài hành động của Indonesia năm 2018 dựa trên bộ tiểu thuyết Wiro Sableng của Bastian Tito.
Phim là sự hợp tác đầu tiên giữa một hãng phim của Indonesia với Fox International Productions.
Một đoạn teaser của bộ phim đã được phát hành vào ngày 21 tháng 12 năm 2017 trên khắp các rạp chiếu phim ở Indonesia. Đoạn trailer đầu tiên được phát hành vào ngày 12 tháng 5 năm 2018 và ra mắt trên ứng dụng truyền thông LINE.
Brown giả định suất phản chiếu của là 0,09 và cường độ l 4,5, dẫn đến đường kính ước tính của vật thể này là 574 km.Tuy nhiên, bởi vì suất phản chiếu của chưa xác định được và có cường độ tuyệt đối ước tính là nên đường kính của nó có thể dễ dàng rơi vào khoảng 414 đến 933 km đối với suất phản chiếu giả định tương ứng là 0,25 và 0,05.
---------------------
Given the context information and not prior knowledge

---

**`Source Node 1/2`**

**Node ID:** 5314078d-5d18-4b21-820f-2859b6aa06ac<br>**Similarity:** 0.8625134035412558<br>**Text:** 212 Warrior (tên tiếng Việt: Chiến binh 212) (còn được biến đến với tên gọi khác là Wiro Sableng 212 và Wiro Sableng Pendekar Kapak Maut Naga Geni 212) là một bộ phim hài hành động của Indonesia năm 2018 dựa trên bộ tiểu thuyết Wiro Sableng của Bastian Tito. Phim là sự hợp tác đầu tiên giữa một hãng phim của Indonesia với Fox International Productions.
Một đoạn teaser của bộ phim đã được phát hành vào ngày 21 tháng 12 năm 2017 trên khắp các rạp chiếu phim ở Indonesia. Đoạn trailer đầu tiên đư...<br>**Metadata:** {'chunk_size': 256}<br>

---

**`Source Node 2/2`**

**Node ID:** c3ac9a82-f340-4ff0-a7a0-1e5087d25f7e<br>**Similarity:** 0.8413752173503093<br>**Text:** Brown giả định suất phản chiếu của là 0,09 và cường độ l 4,5, dẫn đến đường kính ước tính của vật thể này là 574 km.Tuy nhiên, bởi vì suất phản chiếu của chưa xác định được và có cường độ tuyệt đối ước tính là nên đường kính của nó có thể dễ dàng rơi vào khoảng 414 đến 933 km đối với suất phản chiếu giả định tương ứng là 0,25 và 0,05.

212 Warrior (tên tiếng Việt: Chiến binh 212) (còn được biến đến với tên gọi khác là Wiro Sableng 212 và Wiro Sableng Pendekar Kapak Maut Naga Geni 212) là một ...<br>**Metadata:** {'chunk_size': 256}<br>

## Evaluate

In [17]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.evaluation import (
    DatasetGenerator,
    FaithfulnessEvaluator,
    RelevancyEvaluator
)

import openai
import time

openai.api_key = 'OPENAI-API-KEY'

# eval_documents = docs[:20]
data_generator = DatasetGenerator.from_documents(docs)
eval_questions = data_generator.generate_questions_from_nodes(num = 10)

# We will use GPT-4 for evaluating the responses
gpt4 = OpenAI(temperature=0, model="gpt-3.5-turbo-1106")

# Define service context for GPT-4 for evaluation
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

# Define Faithfulness and Relevancy Evaluators which are based on GPT-4
faithfulness_gpt4 = FaithfulnessEvaluator(service_context=service_context_gpt4)
relevancy_gpt4 = RelevancyEvaluator(service_context=service_context_gpt4)

/opt/conda/lib/python3.10/site-packages/llama_index/evaluation/dataset_generation.py:184: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-bdC6gldZoL3tkUvEBiH1WD3I on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
def evaluate_response_time_and_accuracy(query_engine):
    total_response_time = 0
    total_faithfulness = 0
    total_relevancy = 0

    num_questions = len(eval_questions)

    for question in eval_questions:
        start_time = time.time()
        response_vector = query_engine.query(question)
        elapsed_time = time.time() - start_time
        
        faithfulness_result = faithfulness_gpt4.evaluate_response(
            response=response_vector
        ).passing
        
        relevancy_result = relevancy_gpt4.evaluate_response(
            query=question, response=response_vector
        ).passing

        total_response_time += elapsed_time
        total_faithfulness += faithfulness_result
        total_relevancy += relevancy_result

    average_response_time = total_response_time / num_questions
    average_faithfulness = total_faithfulness / num_questions
    average_relevancy = total_relevancy / num_questions

    return average_response_time, average_faithfulness, average_relevancy

# Iterate over different chunk sizes to evaluate the metrics to help fix the chunk size.
for query_engine in query_engines:
  avg_time, avg_faithfulness, avg_relevancy = evaluate_response_time_and_accuracy(query_engine)
  print(f"Chunk size {chunk_size} - Average Response time: {avg_time:.2f}s, Average Faithfulness: {avg_faithfulness:.2f}, Average Relevancy: {avg_relevancy:.2f}")